In [5]:
import base64
import hashlib
import io
import json
import os
import re
import sys
import time
import urllib.parse
from pathlib import Path
from tempfile import NamedTemporaryFile
from typing import Dict, List
from urllib.parse import urlparse
from zipfile import ZipFile

import pandas as pd
import requests
from dotenv import load_dotenv
from google.cloud import storage
from google.cloud.storage import Client, transfer_manager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

sys.path.append("..")
from function.scraper_wedding_wire import process_venues_and_photos


In [ ]:
# Set up Chrome options
options = Options()
options.add_argument("--headless")  # Run browser in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome driver with correct version
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)

In [2]:
def main():
    bucket_name = "wedding-venues-001"

    try:
        print("Starting image download and upload process...")
        results = process_venues_and_photos(bucket_name)
        print("\nProcessing completed!")
        print(f"Successfully processed: {results['success']} images")
        print(f"Skipped (already downloaded): {results['skipped']} images")
        print(f"Failed to process: {results['failed']} images")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


if __name__ == "__main__":
    main()

Starting image download and upload process...

Processing venue: 94th Aero Squadron Restaurant_

Processing venue: a.o.c. Brentwood

Processing venue: Ace Hotel & Swim Club

Processing venue: Agua Hedionda Nature Center

Processing venue: Carousel House Santa Barbara

Processing venue: Alcazar Palm Springs

Processing venue: The Bank Ballroom

Processing venue: Aliso Viejo by Wedgewood Weddings

Processing venue: Aliso Viejo Country Club
Skipping already downloaded image for Aliso Viejo Country Club: Photo 1
Skipping already downloaded image for Aliso Viejo Country Club: Photo 2
Skipping already downloaded image for Aliso Viejo Country Club: Photo 3
Skipping already downloaded image for Aliso Viejo Country Club: Photo 4
Skipping already downloaded image for Aliso Viejo Country Club: Photo 5
Skipping already downloaded image for Aliso Viejo Country Club: Photo 6
Skipping already downloaded image for Aliso Viejo Country Club: Photo 7

Processing venue: Almansor Court
Skipping already dow

In [2]:
import sys

sys.path.append("..")

from function.retriever import initialize_retriever

retriever = initialize_retriever()
docs = retriever.vectorstore.docstore._dict

Created new FAISS index in 


In [3]:
docs

{'10fd68ef-1e1c-4c1a-bb2b-f421ea80c068': Document(metadata={'company': 'dummy', 'type': 'text'}, page_content='dummy document')}

In [ ]:
# from google.cloud import storage
# from pathlib import Path


# def move_venue_images(bucket_name: str):
#     """
#     Move images from /processed/venue_name/figures/ to /processed/adobe_extracted/venue_name/figures/
#     """
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)

#     # List all blobs in the old structure
#     old_prefix = "processed/"
#     new_prefix = "processed/adobe_extracted/"

#     # Keep track of statistics
#     moved = 0
#     errors = 0

#     for blob in bucket.list_blobs(prefix=old_prefix):
#         # Skip if it's already in adobe_extracted
#         if "adobe_extracted" in blob.name:
#             continue

#         # Check if this is a figure file (in figures directory)
#         if not "/figures/" in blob.name:
#             continue

#         try:
#             # Construct the new path
#             # From: processed/venue_name/figures/image.jpg
#             # To: processed/adobe_extracted/venue_name/figures/image.jpg
#             old_path = blob.name
#             new_path = old_path.replace(old_prefix, new_prefix)

#             # Copy to new location
#             new_blob = bucket.copy_blob(blob, bucket, new_path)

#             # Delete the old blob
#             blob.delete()

#             moved += 1
#             if moved % 100 == 0:  # Progress update every 100 files
#                 print(f"Moved {moved} files...")

#         except Exception as e:
#             print(f"Error moving {blob.name}: {str(e)}")
#             errors += 1

#     print(f"\nComplete! Moved {moved} files with {errors} errors")


# # Run the function
# bucket_name = "wedding-venues-001"
# move_venue_images(bucket_name)